# Information Extraction from CVs

## Importing libraries

In [1]:
## General utitlies
from pdfminer3.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer3.converter import TextConverter
from pdfminer3.layout import LAParams
from pdfminer3.pdfpage import PDFPage
from io import BytesIO
from io import StringIO

import os
import pandas as pd
import numpy as np
import datetime

## Keyword Search Utility 
from flashtext import KeywordProcessor

## Resume/CV extraction utilties
from getPhone import getPhone
from getEmail import getEmail
from getName import getName
from getExperience import *
from getEducation import *
from getSkills import *

## Keyword Search

In [2]:
def clean_text_func(text):
        text = str(text)
        text = text.lower()
        text = re.sub(r'[^\x00-\x7F]+',' ', text)
        text = text.replace("\n", " ")
        translator = re.compile('[%s]' % re.escape(string.punctuation))
        text = translator.sub(' ', text)
        text = re.sub(' +', ' ', text)
        text = " ".join(text.split())
        text = text.strip()
        return text

class basic_keyword_extract:
    """Basic keyword extractor class using the keyword processor by Flashtext"""
    def __init__(self, keyword_list, case_sensitive=False):
        self.keyword_list = keyword_list
        self.case_sensitive = case_sensitive
        self.keyword_processor = self._initialize_basic_keyword_processor()
   
    def _initialize_basic_keyword_processor(self):
        kp = KeywordProcessor(case_sensitive=self.case_sensitive)
        kp.add_keywords_from_list(self.keyword_list)
        return kp
    def extract_from_text(self, raw_text):
        if self.case_sensitive:
            text = raw_text
        else:
            text = raw_text.lower().strip()
            clean_text = clean_text_func(text)
            
        clean_extracted_keywords_ls = self.keyword_processor.extract_keywords(clean_text, span_info=True)
#         output_ls = list(set([i[0] for i in clean_extracted_keywords_ls]))
        output_ls = [{'keyword':i[0], 'start_idx':i[1], 'end_idx':i[2]} for i in clean_extracted_keywords_ls]
#         output_ls = list({i['keyword']:i for i in reversed(output_ls)}.values()) ## Remove dups
        if len(output_ls) == 0:
            return []
        else:
            return output_ls

## Convert PDF to text

In [3]:
def convertPDFToText_String(path):
    try:
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr,codec=codec,laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos=set()
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)
        fp.close()
        device.close()
        string = retstr.getvalue()
        retstr.close()
        return string
    except Exception as e:
        print(traceback.format_exc())
        print(e)
        return "Fail"

In [4]:
def convertPDFToText_Bytes(path):
    try:
        rsrcmgr = PDFResourceManager()
        retstr = BytesIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr,codec=codec,laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos=set()
        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
            interpreter.process_page(page)
        fp.close()
        device.close()
        string = retstr.getvalue()
        retstr.close()
        return string
    except Exception as e:
        print(traceback.format_exc())
        print(e)
        return "Fail"

## List of Names

In [5]:
# Reads Indian Names from the file, reduce all to lower case for easy comparision [Name lists]
indianNames = open("IndianNames.txt", "r").read().lower()
indianNames = set(indianNames.split())

## List of Nouns

In [6]:
with open("nounlist.txt") as word_file:
    english_words = set(word.strip().lower() for word in word_file)

## List of Companies 

In [7]:
file = "companies_list.txt"
file = open(file, "r", encoding='utf-8')

## Create Company List
COMPANIES_LIST = [line.strip().lower() for line in file]

## Initialize Keyword Processor:
COMPANY_KEYWORDS_SEARCH = basic_keyword_extract(COMPANIES_LIST, case_sensitive=False)

## List of Job Titles

In [8]:
file = "job_titles_list.txt"
file = open(file, "r", encoding='utf-8')

## Create Title List
TITLE_LIST = [line.strip().lower() for line in file]

## Initialize Keyword Processor:
TITLE_KEYWORDS_SEARCH = basic_keyword_extract(TITLE_LIST, case_sensitive=False)

## List of Time Period

In [9]:
start_year = 1947
end_year = int(datetime.datetime.now().year)
year_ls = [str(i) for i in np.arange(start_year, end_year)]

DATE_KEYWORDS_LS = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec',
                    'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november',
                    'december',
                    'date', 'present'] + year_ls

## Initialize Keyword Processor:
DATE_KEYWORDS_SEARCH = basic_keyword_extract(DATE_KEYWORDS_LS, case_sensitive=False)

## List of Colleges

In [10]:
file = "colleges_list.txt"
file = open(file, "r", encoding='utf-8')

COLLEGE_LS = [line.strip().lower() for line in file]

## Initialize Keyword Processor:
COLLEGE_KEYWORDS_SEARCH = basic_keyword_extract(COLLEGE_LS, case_sensitive=False)

## List of Degrees

In [11]:
file = "degree_list.txt"
file = open(file, "r", encoding='utf-8')

DEGREE_LS = [line.strip().lower() for line in file]

## Initialize Keyword Processor:
DEGREE_KEYWORDS_SEARCH = basic_keyword_extract(DEGREE_LS, case_sensitive=False)

## List of Skills

In [12]:
file = "skills_list.txt"
file = open(file, "r", encoding='utf-8')

SKILLS_LS = [line.strip().lower() for line in file]

## Initialize Keyword Processor:
SKILLS_KEYWORDS_SEARCH = basic_keyword_extract(SKILLS_LS, case_sensitive=False)

## Reading all CVs present in PDF format

In [13]:
def get_final_output(directory):
    out_ls = []
    for file in os.listdir(directory):
        if not file.lower().endswith(".pdf"):
            continue
        name = getName(convertPDFToText_Bytes(os.path.join(directory,file)),indianNames,english_words)
        email = getEmail(convertPDFToText_String(os.path.join(directory,file)))
        phone = getPhone(convertPDFToText_String(os.path.join(directory,file)))
        company = getCompanyNames(convertPDFToText_String(os.path.join(directory,file)), COMPANY_KEYWORDS_SEARCH)
        title = getJobTitles(convertPDFToText_String(os.path.join(directory,file)), TITLE_KEYWORDS_SEARCH)
        period = getPeriod(convertPDFToText_String(os.path.join(directory,file)),DATE_KEYWORDS_SEARCH)
        experience = getExperience(convertPDFToText_String(os.path.join(directory,file)), company, title, period)
        college = getCompanyNames(convertPDFToText_String(os.path.join(directory,file)), COLLEGE_KEYWORDS_SEARCH)
        degree = getJobTitles(convertPDFToText_String(os.path.join(directory,file)), DEGREE_KEYWORDS_SEARCH)
        education = getEducation(convertPDFToText_String(os.path.join(directory,file)), college, degree)
        skills = getSkills(convertPDFToText_String(os.path.join(directory,file)), SKILLS_KEYWORDS_SEARCH)
        print(100*"=")
        print(">> File : ", file)
        print(">> Name : ", name)
        print(">> Email : ", email)
        print(">> Phone : ", phone)
        print("\n")
        print(">> Companies : ", list(set([i['keyword'].upper() for i in company])))
        print(">> Title : ", list(set([i['keyword'].upper() for i in title])))
        print(">> Period :", list(set([i['period'].upper() for i in period])))
        print(">> Experience: ", [{k: v.upper() for k, v in d.items() if k == 'company_name' or k == 'title' or k == 'period'} for d in experience])
        print("\n")
        print(">> Colleges : ", list(set([i['keyword'].upper() for i in college])))
        print(">> Degree : ", list(set([i['keyword'].upper() for i in degree])))
        print(">> Education: ", [{k: v.upper() for k, v in d.items() if k == 'college' or k == 'degree'} for d in education])
        print("\n")
        print(">> Skills: ", list(set([i['keyword'].upper() for i in skills])))
        print(100*"=")

        ## Saving in list:
        out_ls.append([file, name, email, phone, list(set([i['keyword'].upper() for i in company])), 
                      list(set([i['keyword'].upper() for i in title])),
                     list(set([i['period'].upper() for i in period])),
                     [{k: v.upper() for k, v in d.items() if k == 'company_name' or k == 'title' or k == 'period'} for d in experience],
                     list(set([i['keyword'].upper() for i in college])),
                     list(set([i['keyword'].upper() for i in degree])),
                     [{k: v.upper() for k, v in d.items() if k == 'college' or k == 'degree'} for d in education],
                     list(set([i['keyword'].upper() for i in skills]))])

        print("\n")

    ## Create a dataframe
    out_df = pd.DataFrame(out_ls, columns = ['FILENAME', 'NAME', 'EMAIL', 'PHONE NO', 'COMPANIES', 'JOB TITLE', 'TIME PERIOD',
           'EXPERIENCE', 'COLLEGE', 'DEGREE', 'EDUCATION', 'SKILLS'])
    
    return out_df

In [14]:
directory = r'C:\Users\admin\OneDrive - True North Managers LLP\Desktop\TrueNorth\1. Projects\1. 3A Team Project\6. CV Extraction\1. Shared\CVs'

out_df = get_final_output(directory)

>> File :  Abha Doshi- CV.pdf
>> Name :  Abha Doshi
>> Email :  ['abhadoshi@gmail.com']
>> Phone :  ['+91-98207-08442']


>> Companies :  ['SHAH ASSOCIATES', 'KPMG', 'GODREJ HERSHEY', 'GODREJ GROUP', 'AMBIT PRAGMA ADVISORS LLP', 'THE HERSHEY COMPANY', 'FINE LINE CIRCUITS LIMITED']
>> Title :  ['VICE PRESIDENT', 'FINANCE PROFESSIONAL', 'PROJECT MANAGER', 'ASSISTANT MANAGER', 'SENIOR CONSULTANT']
>> Period : ['JUN 2009 JUL 2011', 'AUG 2009 APR 2011', 'R STAHL AG WALDENBURG GERMANY INDUSTRY MANUFACTURING MAY JUN 2007', 'JUN 2014 TO DATE', 'VOLUNTARY ENGAGEMENT YOUTH TEAM LEADER SINCE 2005 AT INITIATIVES OF CHANGE AN INTERNATIONAL ORGANISATION WHICH', 'NA SHAH ASSOCIATES MUMBAI INDIA INDUSTRY FINANCIAL ADVISORY MAY 2004', 'JUN 2004 MAY 2009', 'UP TO MAY 2004', 'FINE LINE CIRCUITS LIMITED MUMBAI INDIA INDUSTRY MANUFACTURING MAY JUN 2005', 'NOV 2012 DEC 2013', 'SEP 2011 SEP 2012', 'KSPG AG NECKARSULM GERMANY INDUSTRY AUTOMOTIVE JUL SEP 2012']
>> Experience:  [{'company_name': 'THE HERSHEY CO

>> File :  Abhinaw Kumar- CV.pdf
>> Name :  Abhinaw Kumar
>> Email :  ['abhinav.saurabh@gmail.com']
>> Phone :  ['+91-7760052177']


>> Companies :  ['DOCTORS CIRCLE']
>> Title :  ['SENIOR SOFTWARE ENGINEER', 'PRODUCT OWNER', 'TEAM LEAD', 'PRODUCT MANAGER', 'HEAD OF PRODUCTS', 'DIGITAL MARKETING PROGRAM MANAGER']
>> Period : ['2013 2013', 'ADMITTED THROUGH IIT JOINT ENTRANCE EXAMINATION 2002 AIR 2986 MAJOR NATURAL LANGUAGE', 'AUG 2006 SEP 2008', '2002 2006', 'CUSTOMERS CURRENT ORGANIC USER GROWTH RATE IS 2000 NEW USERS PER MONTH', 'JUL 2013 FEB 2014', '2012 2013', 'MAR 2014 MARCH 2015', 'APR 2011 AUG 2012', 'JUNE 2010 MARCH 2011', 'SEP 2008 MAY 2010', 'JULY 2015 PRESENT']
>> Experience:  [{'company_name': 'DOCTORS CIRCLE', 'title': 'HEAD OF PRODUCTS', 'period': ''}]


>> Colleges :  ['INDIAN INSTITUTE OF INFORMATION TECHNOLOGY ALLAHABAD', 'INDIAN INSTITUTE OF MANAGEMENT INDORE']
>> Degree :  ['MASTER OF BUSINESS ADMINISTRATION', 'MBA', 'B TECH']
>> Education:  [{'college': 'INDIAN INST

>> File :  Abhishek Shailendra Agarwal- CV.pdf
>> Name :  Abhishek Shailendra Agrawal
>> Email :  ['abhi01agr@gmail.com']
>> Phone :  ['+91 8016907420']


>> Companies :  ['DEUTSCHE BANK', 'ERNST YOUNG']
>> Title :  ['PRINCIPAL SECRETARY', 'VICE PRESIDENT', 'SECRETARY', 'ANALYST']
>> Period : ['2013', '2016', 'MAY JUN 16', '2012 2013', '2008', '2015 16', 'MAR 16', 'JUN 16 PRESENT', 'AUG 15', 'HEADED PROJECT CLEANLINESS ROTARACT CLUB CONDUCTED CLEANLINESS DRIVES TO ACHIEVE THE AIM OF A CLEAN INDIA 2012 13', '2015 2016', '2007', 'JULY 13 APRIL 16', '2014 2016', '2010', 'OCT 12 APR 16', 'CONDUCTED PUNE AND NAGPUR LEG OF NATIONAL LEVEL ENTREPRENEURSHIP AWARENESS DRIVE 2013 MANAGING PUBLICITY', 'MAY JULY 13', 'MAY JULY 15', 'MAY 15 APR 16', 'AUG 12 JAN 13', '2012', '2014 15', 'MAY JULY 14']
>> Experience:  [{'company_name': 'DEUTSCHE BANK', 'title': 'ANALYST', 'period': '2010'}, {'company_name': 'ERNST YOUNG', 'title': '', 'period': ''}]


>> Colleges :  ['IIT KHARAGPUR', 'INDIAN INSTITUTE 

>> File :  Adarsh V K- CV.pdf
>> Name :  Adarsh V K
>> Email :  ['adarsh.vk81@gmail.com']
>> Phone :  ['+91 9672999098']


>> Companies :  ['TRIDENT HILTON', 'TRIDENT HOTELS']
>> Title :  ['MANAGER', 'FRONT OFFICE SUPERVISOR', 'EXECUTIVE CHEF', 'FRONT OFFICE MANAGER', 'MYSTERY SHOPPER', 'GENERAL MANAGER', 'ASSISTANT MANAGER']
>> Period : ['PART OF THE NATIONWIDE MOVEMENT IN 2004', 'AUG 2010 JULY 2013', 'MAY 2016 MAR 2017', 'CHENNAI 1999 2002', 'MAY 2002 MAY 2005', 'CHARGE IN THE ABSENCE OF A MANGER FROM OCTOBER 2003 UNTIL MAY 2005', 'AUG 2013 APR 2016', 'JULY 2007 JULY 2010', 'NEW DELHI 2005 2007']
>> Experience:  [{'company_name': 'TRIDENT HILTON', 'title': 'FRONT OFFICE SUPERVISOR', 'period': 'MAY 2002 MAY 2005'}, {'company_name': 'TRIDENT HOTELS', 'title': '', 'period': ''}]


>> Colleges :  ['THE OBEROI CENTRE OF LEARNING AND DEVELOPMENT', 'CANAN SCHOOL OF CATERING AND HOTEL MANAGEMENT']
>> Degree :  ['DIPLOMA IN HOSPITALITY MANAGEMENT', 'POST GRADUATE DIPLOMA IN HOSPITALITY MANAGE

>> File :  ADITYA CHAMARIA- CV.pdf
>> Name :  Aditya Chamaria
>> Email :  ['adityachamaria@gmail.com']
>> Phone :  ['+91-8527399568']


>> Companies :  ['RECKITT BENCKISER', 'PARAS PHARMACEUTICALS']
>> Title :  ['MANAGER', 'MANAGEMENT ASSISTANT', 'SECRETARY', 'COMPANY SECRETARY', 'CHARTERED ACCOUNTANT', 'PRESIDENT', 'SENIOR MANAGER', 'FACTORY MANAGER', 'COMMERCIAL MANAGER', 'FINANCE PROFESSIONAL']
>> Period : ['JAN 2009 MAY 2011', 'DIPLOMA IN IFRS ACCA UK 2012', 'HEALTHCARE WITH REVENUES OF USD 12 4 BILLION IN 2015 CORPORATE OFFICE GURGAON SINCE DEC', 'MAY 2011 JAN 2013', 'COMPANY SECRETARY EXECUTIVE CLEARED IN DEC 2003', 'DEC 2007 JAN 2009', 'STERLITE OPTICAL TECHNOLOGIES LTD VEDANTA GROUP AURANGABAD FACTORY SEP 04 TO AUG 05', 'COMMODITIES HYDERABAD AUG 05 TO DEC 07', 'FINANCE CONTROLLER MANUFACTURING SOA MIDDLE EAST OCT 2015 TILL DATE', 'ENSURED FAVORABLE RESULT IN THE INCOME TAX ASSESSMENT FOR THE AY 2003 04 RELATED TO', 'CHARTERED ACCOUNTANT CLEARED IN MAY 2004 ALL STAGES FIRST ATT

...

>> File :  Vikas Lal- CV.pdf
>> Name :  Vikas Lal
>> Email :  []
>> Phone :  []


>> Companies :  ['LG ELECTRONICS', 'WHIRLPOOL', 'GLOBAL TECHNOLOGIES', 'COCA COLA']
>> Title :  ['MANAGER', 'FINANCE DIRECTOR', 'CHIEF MARKETING OFFICER', 'CHARTERED ACCOUNTANT', 'GENERAL MANAGER', 'CFO', 'SENIOR MANAGER', 'MANAGING DIRECTOR', 'FINANCIAL SERVICES MANAGER', 'CONSULTANT', 'DIRECTOR', 'FINANCE PROFESSIONAL', 'HEAD OF FINANCE']
>> Period : ['MARCH 2007 OCTOBER 2007 8 MONTHS', '1999 2000 1 YEAR', 'B COM H COMMERCE 1994 1997', '2000 2005 5 YEARS', 'NOVEMBER 2009 PRESENT 6 YEARS 9 MONTHS', 'OCTOBER 2007 DECEMBER 2009 2 YEARS 3 MONTHS', 'JUNE 2005 MARCH 2007 1 YEAR 10 MONTHS', '7 7 2016 VIEW BY ROLI SANGHI AKA CAROLINA', 'CHARTERED ACCOUNTANT ACCOUNTANCY 1996 1999']
>> Experience:  [{'company_name': 'WHIRLPOOL', 'title': 'SENIOR MANAGER', 'period': '2000 2005 5 YEARS'}, {'company_name': 'LG ELECTRONICS', 'title': 'CHARTERED ACCOUNTANT', 'period': '1999 2000 1 YEAR'}, {'company_name': 'COCA COLA',

>> File :  Vinayak Herlekar- CV (IKYA).pdf
>> Name :  Vinayak Herlekar
>> Email :  ['herlekarv@yahoo.co.uk']
>> Phone :  ['9967604946']


>> Companies :  ['BAJAJ ELECTRICALS', 'INSTITUTE OF COMPUTER', 'COMPUTER SOFTWARE', 'NISA GROUP OF COMPANIES', 'RELIANCE RETAIL', 'PANACHE GLOBE SOLUTIONS']
>> Title :  ['PARTNER', 'MANAGER', 'REGIONAL MANAGER', 'CO FOUNDER', 'BUSINESS PARTNER', 'FOUNDER', 'DIRECTOR']
>> Period : ['IN MARCH 2000 WAS', 'MASTERS IN COMPUTER MANAGEMENT INDSEARCH PUNE UNIVERSITY 2002', 'FEB 2006 TO SEPT', '2010', 'SINCE SEPT 2015', 'DATE OF BIRTH 19TH NOVEMBER 1974', '2006', 'MAY 1999 TO MAR', 'LEARNING 2005', '2008', '2015', 'B SC GEOLOGY FERGUSSON COLLEGE PUNE UNIVERSITY 1995', 'MAR 2005 TO FEB', 'COMPUTER STUDIES RESEARCH 1998', 'APRIL 2008 TO OCT', 'SEPT 2006 TO APRIL', '2005', 'NOV 2010 AUG']
>> Experience:  [{'company_name': 'RELIANCE RETAIL', 'title': 'MANAGER', 'period': '2006'}, {'company_name': 'PANACHE GLOBE SOLUTIONS', 'title': 'FOUNDER', 'period': ''}, {'com

>> File :  Vipin K- CV.pdf
>> Name :  Vipin K
>> Email :  []
>> Phone :  []


>> Companies :  ['WHIRLPOOL OF INDIA LTD', 'YUM RESTAURANTS INDIA', 'FLIPKART']
>> Title :  ['ASSOCIATE DIRECTOR', 'MANAGER', 'EFFICIENCY EXPERT', 'FINANCIAL MANAGEMENT', 'PEOPLESOFT FINANCIALS', 'MANAGER FINANCIAL PLANNING', 'SR MANAGER', 'CFO']
>> Period : ['AUG 08 SEP 09', '20 MAY 1981', 'NOV 06 JUL 08', 'SEP 15 TILL DATE', 'FINALIZATION FOR THE YEARS 2007 2008', '2001', '2006', 'FEB 13 JUL 14', 'DATE OF BIRTH', 'AUG 11 JAN 13', 'OCT 09 JUL 11', 'EFFICIENCIES IN NEW STORE OPENINGS GENERAL OVERHEADS IN EXCESS OF 4 MN IN 2014 THIS IS NOW BEING', 'AUG 14 AUG 15']
>> Experience:  [{'company_name': 'YUM RESTAURANTS INDIA', 'title': 'SR MANAGER', 'period': 'FINALIZATION FOR THE YEARS 2007 2008'}, {'company_name': 'WHIRLPOOL OF INDIA LTD', 'title': 'MANAGER FINANCIAL PLANNING', 'period': 'SEP 15 TILL DATE'}, {'company_name': 'FLIPKART', 'title': 'ASSOCIATE DIRECTOR', 'period': 'FINALIZATION FOR THE YEARS 2007 200

>> File :  Vishesh Sharma- CV.pdf
>> Name :  Vishesh Sharma
>> Email :  ['visheshsharma4@gmail.com']
>> Phone :  ['+91-983 388 9382']


>> Companies :  ['BOSTON CONSULTING GROUP', 'ERNST YOUNG']
>> Title :  ['SENIOR ASSOCIATE', 'SUMMER INTERN']
>> Period : ['OVERALL COORDINATOR MOOD INDIGO 2013 IIT BOMBAY S ANNUAL CULTURAL FESTIVAL', 'APR 14 MAR 15', '2010', '1ST ELECTRONIC DANCE MUSIC EDM PERFORMANCE REPOSITIONING CONCERTS IN SYNC WITH PRESENT TRENDS 8000 AUDIENCE', 'JUL 15 JUL 17', 'PURSUING TO CLEAR CFA INSTITUTE USA LEVEL I EXAMINATION IN DEC 17', 'FEB 13 MAR 14', 'MAR 12 FEB 13', '2008', '2015', 'MAY 14 JUL 14', 'CORE GROUP MEMBER MARKETING MOOD INDIGO 2012']
>> Experience:  [{'company_name': 'ERNST YOUNG', 'title': 'SUMMER INTERN', 'period': ''}, {'company_name': 'BOSTON CONSULTING GROUP', 'title': 'SENIOR ASSOCIATE', 'period': 'JUL 15 JUL 17'}]


>> Colleges :  ['IIT BOMBAY', 'CFA INSTITUTE']
>> Degree :  ['M TECH', 'B TECH']
>> Education:  [{'college': 'IIT BOMBAY', 'degree': '

>> File :  Yash Bansal- CV.pdf
>> Name :  Arriving
>> Email :  ['bansalyash16@gmail.com']
>> Phone :  ['+91-9619977988']


>> Companies :  ['MAPE ADVISORY GROUP', 'GRANT THORNTON']
>> Title :  ['INTERNSHIP', 'TAX CONSULTANT', 'ANALYST']
>> Period : ['DATE OF BIRTH 16 NOVEMBER 1993', '2013', '2016', 'APPEARING IN JUNE 2018', '2009', '2014', 'SEPT 16 PRESENT', '2011']
>> Experience:  [{'company_name': 'MAPE ADVISORY GROUP', 'title': 'ANALYST', 'period': 'SEPT 16 PRESENT'}, {'company_name': 'GRANT THORNTON', 'title': 'TAX CONSULTANT', 'period': ''}]


>> Colleges :  ['IIT BOMBAY', 'ICAI', 'CFA INSTITUTE']
>> Degree :  []
>> Education:  []


>> Skills:  ['TAX', 'DCF VALUATION', 'PROCESS ENGINEERING', 'TRA', 'NBFC', 'BUILDING', 'CREDIT RATING', 'DUE DILIGENCE', 'MODELS', 'FOUNDATION', 'CRYPTO', 'COMPLETION', 'VALUATION', 'CLASSIFICATION', 'COMPOUNDING', 'SME', 'SCENARIO ANALYSIS', 'INVESTORS', 'POLYMER COMPOUNDING', 'CURRENCY', 'RUBBER', 'PACKAGING', 'BASE', 'LIABILITY', 'POLYPROPYLENE', 'S

In [15]:
out_df.head()

,FILENAME,NAME,EMAIL,PHONE NO,COMPANIES,JOB TITLE,TIME PERIOD,EXPERIENCE,COLLEGE,DEGREE,EDUCATION,SKILLS
0,Abha Doshi- CV.pdf,Abha Doshi,[abhadoshi@gmail.com],[+91-98207-08442],"[SHAH ASSOCIATES, KPMG, GODREJ HERSHEY, GODREJ...","[VICE PRESIDENT, FINANCE PROFESSIONAL, PROJECT...","[JUN 2009 JUL 2011, AUG 2009 APR 2011, R STAHL...","[{'company_name': 'THE HERSHEY COMPANY', 'titl...","[UNIVERSITY OF MUMBAI, INDIAN INSTITUTE OF MAN...","[MASTER OF COMMERCE, MBA, BACHELOR OF COMMERCE...","[{'college': 'UNIVERSITY OF MUMBAI', 'degree':...","[GERMAN, FINANCIAL PLANNING, MULTINATIONAL, DU..."
1,Abhida Ghatame- CV.pdf,Abhidha Ghatame,[e17abhidhag@iimidr.ac.in],[9975059045],"[ANKUR SEEDS, BNP PARIBAS, RELIGARE FINVEST, I...","[CREDIT MANAGER, CONSULTANT, CHARTERED ACCOUNT...","[1999, APRIL 2011 OCTOBER 2014, OCTOBER 2003 M...","[{'company_name': 'RELIGARE FINVEST', 'title':...","[ICAI, IIM INDORE, NAGPUR UNIVERSITY]","[B COM, MBA, CHARTERED ACCOUNTANT]","[{'college': 'NAGPUR UNIVERSITY', 'degree': 'B...","[ANALYTICS, DISBURSEMENT, LOANS, CREDIT, CAPIT..."
2,Abhilash Pillai- CV.pdf,Abhilash Pillai,[],[],"[ITC LIMITED, INTERWOVEN INC, FLIPKART]","[SENIOR SOFTWARE DEVELOPER, SENIOR PRODUCT MAN...","[APRIL 2015 PRESENT, PGDM MARKETING STRATEGY 2...","[{'company_name': 'ITC LIMITED', 'title': 'SAL...","[INDIAN INSTITUTE OF MANAGEMENT LUCKNOW, IIM L...",[PGDM],[{'college': 'INDIAN INSTITUTE OF MANAGEMENT L...,"[WRITING, PHILOSOPHY, BOUNDARIES, APAC, MERCHA..."
3,Abhinaw Kumar- CV.pdf,Abhinaw Kumar,[abhinav.saurabh@gmail.com],[+91-7760052177],[DOCTORS CIRCLE],"[SENIOR SOFTWARE ENGINEER, PRODUCT OWNER, TEAM...","[2013 2013, ADMITTED THROUGH IIT JOINT ENTRANC...","[{'company_name': 'DOCTORS CIRCLE', 'title': '...",[INDIAN INSTITUTE OF INFORMATION TECHNOLOGY AL...,"[MASTER OF BUSINESS ADMINISTRATION, MBA, B TECH]",[{'college': 'INDIAN INSTITUTE OF MANAGEMENT I...,"[OFFLINE MARKETING, FRAMING, MANAGEMENT SYSTEM..."
4,Abhiram K A- CV.pdf,Abhiram K A,[abhiramkatta@gmail.com],[800 888 3081],"[WIPRO LIMITED, HSBC, MYSMARTPRICE, WIPRO, NAT...","[INTERNSHIP, VICE PRESIDENT, BOARD MEMBER, SEN...","[SEP 15 TO DATE, 2000 2003 CHARTERED ACCOUNTAN...","[{'company_name': 'WIPRO LIMITED', 'title': 'S...","[OSMANIA UNIVERSITY, IIM BANGALORE, INSTITUTE ...","[B COM, MBA, POST GRADUATE, CHARTERED ACCOUNTA...","[{'college': 'OSMANIA UNIVERSITY', 'degree': '...","[ANALYTICS, APPAREL, FERTILIZER, TRANSFORMATIO..."
